In [13]:
import emoji
import os
import pandas as pd
import re
import gensim
from gensim.corpora import Dictionary
from gensim.models import LdaMulticore, CoherenceModel
#import pyldavis

import nltk
#nltk.download('omw-1.4')
from nltk.stem import WordNetLemmatizer

In [8]:
df = pd.read_csv('airline_tweets_csvs/AmericanAir.csv.gz', low_memory=False)

In [9]:
df.shape

(785, 15)

In [81]:
full_df['created_at'].max()

'Twitter Web Client'

In [82]:
with open("bookmarks_twitter_airlines.txt", "r") as myfile:
    bookmarks = myfile.read().splitlines()
twitter_airline_handles = [bkmk.split("(")[1].split(")")[0] for bkmk in bookmarks]

In [ ]:
# Bucket airlines into continents/regions

region_to_airline_map = {
    'Africa': [],
    
    'United States': [
        'SouthwestAir',
        'united',
        
    ],
    
    'Latin America and Carribean': [
        'AviancaNAM',
        
    ],
    
    'Central/Eastern Asia': [
        'AsianaAirlines', 
    ],
    
    'Southern/South-eastern Asia': [
        'IndonesiaGaruda', 
        'CebuPacificAir',
        'airasia',
        'flyPAL',
        
    ],
    
    'Western Asia': [
        'etihad',
        
    ],
    
    'Europe': [
        'KLM',
        'British_Airways',
        
        
    ],
    
    'Australia and New Zealand': []
    
}

In [ ]:
# Combine all dataframes into one dataframe

saved_csvs_dir = "airline_tweets_csvs/"
df_columns = [
    'tweet_id', 
    'user',
    'created_at', 'text', 'is_reply',
    'source', 'lang',
    'retweet_count', 'reply_count', 'like_count', 'quote_count',
    'detected_domain_entity_pairs',
    'annots_found', 'hashtags', 'mentions'
]
full_df = pd.DataFrame(columns=df_columns)

for csv_file_path in os.listdir(saved_csvs_dir):
    try:
        new_df = pd.read_csv(saved_csvs_dir + '/' + csv_file_path, low_memory=False)
        full_df = pd.concat([full_df, new_df], axis=0)
    except IsADirectoryError:
        continue
    print(full_df.shape)
    # final shape: (59849, 15)

In [4]:
full_df.to_csv('all_airline_tweets_english.csv.gz', compression='gzip', index=False)

In [5]:
full_df = pd.read_csv('all_airline_tweets_english.csv.gz', low_memory=False)

In [6]:
full_df.head()

,tweet_id,user,created_at,text,is_reply,source,lang,retweet_count,reply_count,like_count,quote_count,detected_domain_entity_pairs,annots_found,hashtags,mentions
0,1156918651838054400,easyJet,2019-08-01T13:24:35.000Z,RT @delightdoodahs: Cracking @easyJet flight f...,0.0,Sprinklr,en,3.0,0,0,0,"{'Brand_EasyJet', 'Unified Twitter Taxonomy_Fa...",{'Place_Berlin'},{'lettheholidaysbegin'},"{'delightdoodahs', 'EDI_Airport', 'easyJet'}"
1,1156918495462023168,easyJet,2019-08-01T13:23:58.000Z,RT @vfxflyer: Cloud surfing with @easyjet from...,0.0,Sprinklr,en,1.0,0,0,0,"{'Brand_EasyJet', 'Unified Twitter Taxonomy_Fa...","{'Other_BUD', 'Organization_BSL'}",{'lovetofly'},"{'easyJet', 'vfxflyer'}"
2,1154809777752629249,easyJet,2019-07-26T17:44:41.000Z,RT @LDNLutonAirport: Trains remain severely di...,0.0,Sprinklr,en,2.0,0,0,0,"{'Brand_EasyJet', 'Interests and Hobbies_Trave...",{'Other_Thameslink'},set(),{'LDNLutonAirport'}
3,1154670861376524289,easyJet,2019-07-26T08:32:40.000Z,It may take us a little longer to reply to you...,0.0,Sprinklr,en,2.0,28,11,1,"{'Brand_EasyJet', 'Unified Twitter Taxonomy_Fa...",NaN,NaN,NaN
4,1154670731613081600,easyJet,2019-07-26T08:32:09.000Z,"Due to adverse weather across the UK, easyJet,...",0.0,Sprinklr,en,6.0,34,31,4,"{'Brand_EasyJet', 'Unified Twitter Taxonomy_Fa...",{'Place_UK'},set(),set()


In [7]:
full_df.shape

(59849, 15)

In [8]:
full_df = full_df.dropna(subset=['user', 'text'])

In [9]:
full_df.shape

(59848, 15)

In [36]:
# I used the following source to discover some common pre-processing steps, 
# and I created my own functions to clean tweets: 
# https://pub.towardsai.net/tweet-topic-modeling-part-2-cleaning-and-preprocessing-tweets-e3a08a8b1770


def clean_tweet(row):
    """Performs removal of emojis, link-related text, 
    retweet-related extra text, and hashtags and mentions.
    Further strips punctuation, extra spaces, and 
    numbers. 
    """
    tweet = row['text']

    
    try:
        # Removing emojis (not in project scope)
        tweet = emoji.replace_emoji(tweet, '')
    

        # Removing link related text
        link_related = [r'http', r'bit.ly/', r'pic.twitter']
        for link_to_clean in link_related:
            tweet = re.sub(fr'{link_to_clean}\S+', '', tweet)
        tweet = tweet.strip("[link]")

        # Removing retweet, mention, or hashtag related text
        rt_mtn_hsh_related = [r'RT\s@', r'@', r'#']
        for rt_mtn_hsh_to_clean in rt_mtn_hsh_related:
            tweet = re.sub(fr'({rt_mtn_hsh_to_clean}[A-Za-z]+[A-Za-z0-9-_]+)', '', tweet)

        # Remove punctuation
        punctuation = '!”$%&\’()*+,-./:;<=>?[\\]^_`{|}~•@'
        tweet = re.sub('[' + punctuation + ']+', ' ', tweet)

        # Remove spaces and numbers
        # tweet = re.sub('\s+', ' ', tweet)
        tweet = re.sub('([0-9]+)', '', tweet)
        tweet = tweet.strip()
        
        return tweet
        
    except TypeError:
        print(tweet)
        print(row['user'])
        print(row['tweet_id'])
        return 'CANNOT TRANSFORM'
        
    


def tokenize_and_lemmatize_tweet(row):
    """Returns list of tokens from tweet after lemmatizing. """
    tweet = row['clean_text']
    lemmatizer = WordNetLemmatizer()
    stopwords = gensim.parsing.preprocessing.STOPWORDS
    
    tweet_tokens_lst = []
    tokens = gensim.utils.simple_preprocess(tweet)
    for token in tokens:
        if token not in stopwords and len(token) > 2:
            tweet_tokens_lst.append(lemmatizer.lemmatize(token, pos='v'))
    
    return ' '.join(tweet_tokens_lst)  # tweet


def create_lemma_token_lst(row):
    tokenized_and_lemmatized_tweet = row['tokenize_lemmatize_text']
    return tokenized_and_lemmatized_tweet.split()


In [15]:
full_df['clean_text'] = full_df.apply(lambda row: clean_tweet(row), axis=1)

In [16]:
full_df['tokenize_lemmatize_text'] = full_df.apply(lambda row: tokenize_and_lemmatize_tweet(row), axis=1)

In [37]:
full_df['lemma_tokens'] = full_df.apply(lambda row: create_lemma_token_lst(row), axis=1)

In [40]:
full_df.head()

,tweet_id,user,created_at,text,is_reply,source,lang,retweet_count,reply_count,like_count,quote_count,detected_domain_entity_pairs,annots_found,hashtags,mentions,clean_text,tokenize_lemmatize_text,lemma_tokens
0,1156918651838054400,easyJet,2019-08-01T13:24:35.000Z,RT @delightdoodahs: Cracking @easyJet flight f...,0.0,Sprinklr,en,3.0,0,0,0,"{'Brand_EasyJet', 'Unified Twitter Taxonomy_Fa...",{'Place_Berlin'},{'lettheholidaysbegin'},"{'delightdoodahs', 'EDI_Airport', 'easyJet'}",Cracking flight from to Berlin this morning,crack flight berlin morning,"[crack, flight, berlin, morning]"
1,1156918495462023168,easyJet,2019-08-01T13:23:58.000Z,RT @vfxflyer: Cloud surfing with @easyjet from...,0.0,Sprinklr,en,1.0,0,0,0,"{'Brand_EasyJet', 'Unified Twitter Taxonomy_Fa...","{'Other_BUD', 'Organization_BSL'}",{'lovetofly'},"{'easyJet', 'vfxflyer'}",Cloud surfing with from BUD to BSL,cloud surf bud bsl,"[cloud, surf, bud, bsl]"
2,1154809777752629249,easyJet,2019-07-26T17:44:41.000Z,RT @LDNLutonAirport: Trains remain severely di...,0.0,Sprinklr,en,2.0,0,0,0,"{'Brand_EasyJet', 'Interests and Hobbies_Trave...",{'Other_Thameslink'},set(),{'LDNLutonAirport'},Trains remain severely disrupted on the Thames...,train remain severely disrupt thameslink line ...,"[train, remain, severely, disrupt, thameslink,..."
3,1154670861376524289,easyJet,2019-07-26T08:32:40.000Z,It may take us a little longer to reply to you...,0.0,Sprinklr,en,2.0,28,11,1,"{'Brand_EasyJet', 'Unified Twitter Taxonomy_Fa...",NaN,NaN,NaN,It may take us a little longer to reply to you...,little longer reply volume message currently r...,"[little, longer, reply, volume, message, curre..."
4,1154670731613081600,easyJet,2019-07-26T08:32:09.000Z,"Due to adverse weather across the UK, easyJet,...",0.0,Sprinklr,en,6.0,34,31,4,"{'Brand_EasyJet', 'Unified Twitter Taxonomy_Fa...",{'Place_UK'},set(),set(),Due to adverse weather across the UK easyJet l...,adverse weather easyjet like airlines expect d...,"[adverse, weather, easyjet, like, airlines, ex..."


In [49]:
# Create dictionary of tokens
# all_tweets_tokenized_lemmatized = [[twt] for twt in full_df['tokenize_lemmatize_text'].values]
# dct = Dictionary(all_tweets_tokenized_lemmatized)
dct = Dictionary(full_df['lemma_tokens'])

In [50]:
print(len(dct))

28256


In [51]:
# Filtering Extremes
dct.filter_extremes(no_below=2, no_above=.99)
print(len(dct))

14371


In [52]:
# Creating a corpus object 
corpus = [dct.doc2bow(d) for d in full_df['lemma_tokens']]

In [53]:
len(corpus)

59848

In [54]:
# Instantiating a Base LDA model 
base_model = LdaMulticore(corpus=corpus, num_topics=5, id2word=dct, workers=12, passes=5)

In [55]:
# Filtering for words 
words = [re.findall(r'"([^"]*)"',t[1]) for t in base_model.print_topics()]

In [56]:
# Create Topics
topics = [' '.join(t[0:10]) for t in words]

In [57]:
# Getting the topics
for id, t in enumerate(topics): 
    print(f"------ Topic {id} ------")
    print(t, end="\n\n")

------ Topic 0 ------
flight travel book check time passengers airport amp update change

------ Topic 1 ------
flight new world air year happy airline book travel today

------ Topic 2 ------
fly flight travel book learn seat swiss visit destination ticket

------ Topic 3 ------
amp today thank fly flight day new team know work

------ Topic 4 ------
flight book new day trip fly city start visit today



In [58]:
# Compute Perplexity
# a measure of how good the model is. lower the better
base_perplexity = base_model.log_perplexity(corpus)
print('\nPerplexity: ', base_perplexity) 



Perplexity:  -7.699162251281118


In [59]:
# Compute Coherence Score
coherence_model = CoherenceModel(model=base_model, texts=full_df['lemma_tokens'], 
                                   dictionary=dct, coherence='c_v')
coherence_lda_model_base = coherence_model.get_coherence()
print('\nCoherence Score: ', coherence_lda_model_base)

/opt/anaconda3/envs/big-data-proj/lib/python3.8/site-packages/gensim/topic_coherence/text_analysis.py:441: DeprecationWarning: The 'warn' method is deprecated, use 'warning' instead
  logger.warn("stats accumulation interrupted; <= %d documents processed", self._num_docs)
Process AccumulatingWorker-47:
Traceback (most recent call last):
  File "/opt/anaconda3/envs/big-data-proj/lib/python3.8/multiprocessing/process.py", line 318, in _bootstrap
    util._exit_function()
  File "/opt/anaconda3/envs/big-data-proj/lib/python3.8/multiprocessing/util.py", line 360, in _exit_function
    _run_finalizers()
  File "/opt/anaconda3/envs/big-data-proj/lib/python3.8/multiprocessing/util.py", line 300, in _run_finalizers
    finalizer()
  File "/opt/anaconda3/envs/big-data-proj/lib/python3.8/multiprocessing/util.py", line 224, in __call__
    res = self._callback(*self._args, **self._kwargs)
  File "/opt/anaconda3/envs/big-data-proj/lib/python3.8/multiprocessing/queues.py", line 195, in _finalize_jo

KeyboardInterrupt: 

In [19]:
#!pip3 install pyLDAvis

  Using cached pyLDAvis-3.3.1.tar.gz (1.7 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.1/133.1 kB 2.3 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... error
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [18 lines of output]
      The 'sklearn' PyPI package is deprecated, use 'scikit-learn'
      rather than 'sklearn' for pip commands.
      
      Here is how to fix this error in the main use cases:
      - use 'pip install scikit-learn' rather than 'pip install sklearn'
      - replace 'sklearn' by 'scikit-learn' in your pip requirements files
        (requirements.txt, setup.py, setup.cfg, Pipfile, etc ...)
      - if the 'sklearn' package is used by one of your dependencies,
        it would be great if you take some time to track which package uses
        

In [ ]:
#Creating Topic Distance Visualization 
# pyLDAvis.enable_notebook()
# pyLDAvis.gensim.prepare(base_model, corpus, id2word)

import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
pyLDAvis.enable_notebook()

# feed the LDA model into the pyLDAvis instance
lda_viz = gensimvis.prepare(base_model, corpus, dct)

In [ ]:
lda_viz

In [20]:
import pyLDAvis

/opt/anaconda3/envs/big-data-proj/lib/python3.8/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload


In [78]:
full_df.head()

,tweet_id,user,created_at,text,is_reply,source,lang,retweet_count,reply_count,like_count,quote_count,detected_domain_entity_pairs,annots_found,hashtags,mentions,clean_text,tokenize_lemmatize_text
0,1156918651838054400,easyJet,2019-08-01T13:24:35.000Z,RT @delightdoodahs: Cracking @easyJet flight f...,0.0,Sprinklr,en,3.0,0,0,0,"{'Brand_EasyJet', 'Unified Twitter Taxonomy_Fa...",{'Place_Berlin'},{'lettheholidaysbegin'},"{'delightdoodahs', 'EDI_Airport', 'easyJet'}",Cracking flight from to Berlin this morning,crack flight berlin morning
1,1156918495462023168,easyJet,2019-08-01T13:23:58.000Z,RT @vfxflyer: Cloud surfing with @easyjet from...,0.0,Sprinklr,en,1.0,0,0,0,"{'Brand_EasyJet', 'Unified Twitter Taxonomy_Fa...","{'Other_BUD', 'Organization_BSL'}",{'lovetofly'},"{'easyJet', 'vfxflyer'}",Cloud surfing with from BUD to BSL,cloud surf bud bsl
2,1154809777752629249,easyJet,2019-07-26T17:44:41.000Z,RT @LDNLutonAirport: Trains remain severely di...,0.0,Sprinklr,en,2.0,0,0,0,"{'Brand_EasyJet', 'Interests and Hobbies_Trave...",{'Other_Thameslink'},set(),{'LDNLutonAirport'},Trains remain severely disrupted on the Thames...,train remain severely disrupt thameslink line ...
3,1154670861376524289,easyJet,2019-07-26T08:32:40.000Z,It may take us a little longer to reply to you...,0.0,Sprinklr,en,2.0,28,11,1,"{'Brand_EasyJet', 'Unified Twitter Taxonomy_Fa...",NaN,NaN,NaN,It may take us a little longer to reply to you...,little longer reply volume message currently r...
4,1154670731613081600,easyJet,2019-07-26T08:32:09.000Z,"Due to adverse weather across the UK, easyJet,...",0.0,Sprinklr,en,6.0,34,31,4,"{'Brand_EasyJet', 'Unified Twitter Taxonomy_Fa...",{'Place_UK'},set(),set(),Due to adverse weather across the UK easyJet l...,adverse weather easyjet like airlines expect d...


### Appendix 

In [2]:
def give_emoji_free_text(text):
    return emoji.get_emoji_regexp().sub(r'', text.decode('utf8'))

In [3]:
example_tweet = 'We hope everyone is enjoying their holiday season — we know this couple is! 🎁💍\nhttps://t.co/EzAVQp4LZj'

In [12]:
emoji.replace_emoji(example_tweet, '')

'We hope everyone is enjoying their holiday season — we know this couple is! \nhttps://t.co/EzAVQp4LZj'

In [44]:
full_df.iloc[0, :]['text']

'RT @delightdoodahs: Cracking @easyJet flight from @EDI_Airport to Berlin this morning #lettheholidaysbegin https://t.co/rJJNXu9RO7'

In [56]:
tweet = full_df.iloc[0, :]['text']
tweet

'RT @delightdoodahs: Cracking @easyJet flight from @EDI_Airport to Berlin this morning #lettheholidaysbegin https://t.co/rJJNXu9RO7'

In [57]:
# Removing emojis (not in project scope)
tweet = emoji.replace_emoji(tweet, '')
tweet

'RT @delightdoodahs: Cracking @easyJet flight from @EDI_Airport to Berlin this morning #lettheholidaysbegin https://t.co/rJJNXu9RO7'

In [58]:
# Removing link related text
link_related = [r'http', r'bit.ly/', r'pic.twitter']
for link_to_clean in link_related:
    tweet = re.sub(fr'{link_to_clean}\S+', '', tweet)
tweet = tweet.strip("[link]")

tweet

'RT @delightdoodahs: Cracking @easyJet flight from @EDI_Airport to Berlin this morning #lettheholidaysbegin '

In [59]:
# Removing retweet, mention, or hashtag related text
rt_mtn_hsh_related = [r'RT\s@', r'@', r'#']
for rt_mtn_hsh_to_clean in rt_mtn_hsh_related:
    tweet = re.sub(fr'({rt_mtn_hsh_to_clean}[A-Za-z]+[A-Za-z0-9-_]+)', '', tweet)
    
tweet

': Cracking  flight from  to Berlin this morning  '

In [60]:
# Remove punctuation
punctuation = '!”$%&\’()*+,-./:;<=>?[\\]^_`{|}~•@'
tweet = re.sub('[' + punctuation + ']+', ' ', tweet)
tweet

'  Cracking  flight from  to Berlin this morning  '

In [62]:
# Remove spaces and numbers
tweet = re.sub('\s+', ' ', tweet)
tweet = re.sub('([0-9]+)', '', tweet)
tweet = tweet.strip()

tweet

'Cracking flight from to Berlin this morning'

In [12]:
example_str = r'http'
fr'{example_str}\S+'

'http\\S+'

In [5]:
r'http\S+'

'http\\S+'

In [ ]:
# Source: https://pub.towardsai.net/tweet-topic-modeling-part-2-cleaning-and-preprocessing-tweets-e3a08a8b1770

def remove_links(tweet):
    """Takes a string and removes web links from it"""
    tweet = re.sub(r'http\S+', '', tweet)   # remove http links
    tweet = re.sub(r'bit.ly/\S+', '', tweet)  # remove bitly links
    tweet = tweet.strip('[link]')   # remove [links]
    tweet = re.sub(r'pic.twitter\S+','', tweet)
    return tweet

def remove_users(tweet):
    """Takes a string and removes retweet and @user information"""
    tweet = re.sub('(RT\s@[A-Za-z]+[A-Za-z0-9-_]+)', '', tweet)  # remove re-tweet
    tweet = re.sub('(@[A-Za-z]+[A-Za-z0-9-_]+)', '', tweet)  # remove tweeted at
    return tweet

def remove_hashtags(tweet):
    """Takes a string and removes any hash tags"""
    tweet = re.sub('(#[A-Za-z]+[A-Za-z0-9-_]+)', '', tweet)  # remove hash tags
    return tweet

def remove_av(tweet):
    """Takes a string and removes AUDIO/VIDEO tags or labels"""
    tweet = re.sub('VIDEO:', '', tweet)  # remove 'VIDEO:' from start of tweet
    tweet = re.sub('AUDIO:', '', tweet)  # remove 'AUDIO:' from start of tweet
    return tweet

def tokenize(tweet):
    """Returns tokenized representation of words in lemma form excluding stopwords"""
    result = []
    for token in gensim.utils.simple_preprocess(tweet):
        if token not in gensim.parsing.preprocessing.STOPWORDS \
                and len(token) > 2:  # drops words with less than 3 characters
            result.append(lemmatize(token))
    return result

def lemmatize(token):
    """Returns lemmatization of a token"""
    return WordNetLemmatizer().lemmatize(token, pos='v')

def preprocess_tweet(tweet, lem_and_token=False):
    """Main master function to clean tweets, stripping noisy characters, and tokenizing use lemmatization"""
    tweet = remove_users(tweet)
    tweet = remove_links(tweet)
    tweet = remove_hashtags(tweet)
    tweet = remove_av(tweet)
    tweet = tweet.lower()  # lower case
    tweet = re.sub('[' + punctuation + ']+', ' ', tweet)  # strip punctuation
    tweet = re.sub('\s+', ' ', tweet)  # remove double spacing
    tweet = re.sub('([0-9]+)', '', tweet)  # remove numbers
    if lem_and_token:
        tweet_token_list = tokenize(tweet)  # apply lemmatization and tokenization
        tweet = ' '.join(tweet_token_list)
    return tweet

In [22]:
path_csv = "airline_tweets_csvs/AerLingus.csv.gz"
df = pd.read_csv(path_csv, low_memory=False)